In [20]:
import cv2
import mediapipe as mp
import time
import numpy as np
import PoseModele as pm

##############################

##############################

cap = cv2.VideoCapture("curls.mp4")
pTime = 0

detector = pm.poseDetector()
count = 0
dir = 0

while 1:
    ret, img = cap.read()
    if ret == False:
        break

    img = detector.findPose(img, False)
    lmList = detector.findPosition(img, draw= False)
    if len(lmList) != 0:
        # Right Arm
        angle = detector.finAngle(img, 12, 14, 16)
        # # Left Arm
        # detector.finAngle(img, 11, 13, 15)
        per = 100 - np.interp(angle, (50, 170), (0, 100))
        bar = np.interp(angle, (55, 170), (550, 715))

        #check for the dumblell curls
        color = (255, 0, 255)
        if per == 0:
            color = (0, 0, 255)
            if dir == 0:
                count += 0.5
                dir = 1

        if per == 100:
            color = (0, 255, 0)
            if dir == 1:
                count += 0.5
                dir = 0

        # Draw bar
        cv2.rectangle(img, (320, 550), (370, 715), color, 3)
        cv2.rectangle(img, (320, int(bar)), (370, 715), color, cv2.FILLED)
        cv2.putText(img, f"{int(per)}%", (320, 540), cv2.FONT_HERSHEY_PLAIN,
                    2, color, 3)

        # Draw count
        cv2.rectangle(img, (0, 550), (200, 715), (0, 255, 0), cv2.FILLED)
        cv2.putText(img, f"{int(count)}", (55, 680), cv2.FONT_HERSHEY_PLAIN,
                    8, (255, 0, 0), 10)

        # print(per, dir)
        # cv2.putText(img, f"{int(count)}", (50, 100), cv2.FONT_HERSHEY_PLAIN,
        #             5, (255, 0, 0), 5)
        
    
    cTime = time.time()
    fps = 1/(cTime - pTime)
    pTime = cTime

    cv2.putText(img, f"FPS: {int(fps)}", (30, 50), cv2.FONT_HERSHEY_PLAIN,
                    2, (255, 0, 0), 3)

    cv2.imshow("img", img)
    if cv2.waitKey(10) == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

In [3]:
import cv2
import mediapipe as mp
import time
import cvzone

class poseDetector():
    def __init__(self, mode = False, upBody = False, smooth = True,
                 detectionCon = 0.5, trackCon = 0.5):
        self.mode = mode
        self.upBody = upBody
        self.smooth = smooth
        self.detectionCon = detectionCon
        self.trackCon = trackCon

        self.mpDraw = mp.solutions.drawing_utils
        self.mpPose = mp.solutions.pose
        # self.pose = self.mpPose.Pose(self.mode, self.upBody, self.smooth,
        #                              self.detectionCon, self.trackCon)
        self.pose = self.mpPose.Pose(
            static_image_mode=self.mode,
            enable_segmentation=self.upBody,  # @TODO:
            smooth_segmentation=self.smooth, 
            min_detection_confidence=self.detectionCon, 
            min_tracking_confidence=self.trackCon)
    
    def findPose(self, img, draw = True):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.pose.process(imgRGB)
        if self.results.pose_landmarks:
            if draw:
                self.mpDraw.draw_landmarks(img, self.results.pose_landmarks, self.mpPose.POSE_CONNECTIONS)
        return img   
    
    def findPosition(self, img, draw = True):
        self.lmList = []
        if self.results.pose_landmarks:
            for id, lm in enumerate(self.results.pose_landmarks.landmark):
                h, w, _ = img.shape
                cx, cy = int(lm.x*w), int(lm.y*h)
                self.lmList.append([id, cx, cy])
                if draw:
                    cv2.circle(img, (cx, cy), 5, (255,0,0), cv2.FILLED)
        return self.lmList
    
    def finAngle(self, img, p1, p2, p3, draw = True):
        x1, y1 = self.lmList[p1][1:]
        x2, y2 = self.lmList[p2][1:]
        x3, y3 = self.lmList[p3][1:]

        if draw:
            cv2.line(img, (x1, y1), (x2, y2), (255, 255, 255), 3)
            cv2.line(img, (x3, y3), (x2, y2), (255, 255, 255), 3)
            cv2.circle(img, (x1, y1), 10, (0, 0, 255), cv2.FILLED)
            cv2.circle(img, (x1, y1), 15, (0, 0, 255), 2)
            cv2.circle(img, (x2, y2), 10, (0, 0, 255), cv2.FILLED)
            cv2.circle(img, (x2, y2), 15, (0, 0, 255), 2)
            cv2.circle(img, (x3, y3), 10, (0, 0, 255), cv2.FILLED)
            cv2.circle(img, (x3, y3), 15, (0, 0, 255), 2)



def main():
    cap = cv2.VideoCapture("curls.mp4")
    pTime = 0
    cTime = 0

    detector = poseDetector()

    while 1:
        ret, img = cap.read()
        if ret == False: 
            cap.release()
            cv2.destroyAllWindows()
            break

        img = detector.findPose(img)
        lmList = detector.findPosition(img, False)
        if len(lmList) != 0:
            cv2.circle(img, (lmList[0][1], lmList[0][2]), 5, (255,0,0), cv2.FILLED)


        cTime = time.time()
        fps = 1/(cTime-pTime)
        pTime = cTime

        cv2.putText(img, str(int(fps)), (70,50), cv2.FONT_HERSHEY_PLAIN, 3
                    , (255, 0, 0), 3)

        cv2.imshow("img", img)
        if cv2.waitKey(10) & 0xFF == ord("q"):
            cap.release()
            cv2.destroyAllWindows()
            break


if __name__ == "__main__":
    main()